In [1]:
import pandas as pd
import numpy as np
import json
import requests, os, sys, sqlite3, re
import yfinance as yf

In [98]:
df = pd.read_csv('./out/catalyst_man.csv').iloc[:1774]
df2 = pd.read_csv("./out/roy_fixed.csv")
df2 = df2.loc[:, df2.columns[1:]].copy()
df2['date'] = pd.to_datetime(df2['date'])
df['date'] = pd.to_datetime(df['date'])

/var/folders/xg/pf7prx2j6mngdfnzgpn5r9980000gn/T/ipykernel_35878/914809477.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df2['date'] = pd.to_datetime(df2['date'])


In [99]:
df.columns = df2.columns
df = df.rename(columns={"label":"label_roy"})
tmp_df = df.merge(df2, on=['ticker', 'disease', 'stage', 'date', 'catalyst'], how='outer')
tmp_df['label'] = tmp_df['label_roy'].combine_first(tmp_df['label'])
tmp_df = tmp_df.drop(columns=['label_roy'], axis=1)

In [101]:
tmp_df.to_csv('../final.csv')

In [8]:
eng = sqlite3.connect('./src/data.db')
tix = pd.read_sql('SELECT DISTINCT ticker FROM catalysts', eng)
df = pd.read_sql('SELECT * FROM catalysts', eng)
tickers = yf.Tickers(' '.join(tix.ticker.tolist()))

In [33]:
names = list()
for t in tix.ticker.tolist():
    try:
        name = tickers.tickers[t].info['longName'].upper()
        name = re.sub(r'[^\w\s]', '', name)
        names.append([t, name])
    except Exception:
        names.append([t, None])

In [4]:
with open('./ref/ticker_to_cik.json') as f:
    d = json.load(f)
f.close()

In [29]:
ticks = pd.DataFrame.from_dict(d, orient='index')
ciks = ticks.loc[ticks.index.isin(tix.ticker)].copy().reset_index().rename(columns={'index': 'ticker', 0: 'cik'})
ciks['cik'] = ciks.cik.str.removeprefix('000')

In [31]:
ciks.to_csv('./out/ciks.csv', index=False)